# Introduction

This notebook presents a benchmark comparison between `power-grid-model` and 
[`pandapower`](https://github.com/e2nIEE/pandapower).
It runs several calculations, measures the calculation time, and compares the results.

## Test Network

The test network is fictionally generated using pre-defined random criteria. It is a radial network as follows:

```

source --- source_node ---| ---line--- node ---line--- node ...   (n_node_per_feeder)
                          |              |              |
                          |            load            load ...
                          |
                          | ---line--- ...
                          | .
                          | .
                          | .
                          | (n_feeder)

```

There is a node which is connected to a source (external network). From the source node there are `n_feeder` feeders. For each feeder there are `n_node_per_feeder` nodes, lines, and asymmetric loads. There are in total `n_feeder * n_node_per_feeder + 1` nodes in the network.


## Calculation

The notebook runs a power flow calculation with the same input data in `power-grid-model` and `pandapower`. It runs the following calculations:

* Single calculation with solver initialization.
* Single calculation without solver initialization (using pre-cached internal matrices).
* Time-series calculation
* N-1 calculation

The calculation is run symmetrically and asymmetrically. Both `power-grid-model` and `pandapower` supports asymmetric loads in symmetric calculations: the three-phase load is aggregated into one symmetric load.

We use the Newton-Raphson method for both libraries. In addition, we also use the linear method in `power-grid-model` to see how much performance you can gain in exchange for accuracy.

## Results Comparison

The following results are compared between `power-grid-model` and `pandapower`:

* Per unit voltage of nodes (buses). For asymmetric calculation, it compares the value per phase.
* Loading of the lines.

It only compares the results of the Newton-Raphson method, since the linear method of `power-grid-model` will produce a less accurate result.

## Performance Benchmark

The CPU time is measured for the **calculation part** of the program. The data preparation and model initialization is not measured. Furthermore, the single calculation is benchmarked with and without solver initialization. The former needs to execute connectivity check and initialize internal matrices (for example node admittance matrix). The latter uses the pre-cached connectivity and internal matrices.

# Preparation

## Import Libraries

We import neede libraries here. The fictional network generation and time-series profile generation is in a different Python file
[generate_fictional_dataset.py](./generate_fictional_dataset.py).

In [1]:
import tempfile
import time
from pathlib import Path

import numpy as np
import pandapower as pp
import pandas as pd
import power_grid_model as pgm

from generate_fictional_dataset import generate_fictional_grid, generate_time_series

# output dir for pandapower time-series
output_dir = Path(tempfile.gettempdir()) / "pandapower_time_series"
output_dir.mkdir(exist_ok=True)

## Prepare Tables

The performance comparison table and result deviation table is initialized below.

In [2]:
# summary
summary_df = pd.DataFrame(
    np.zeros(shape=(8, 3), dtype=np.float64),
    columns=['PGM Linear', 'PGM NR', 'PandaPower NR'],
    index=[
        'Symmetric calculation with solver initialization',
        'Symmetric calculation without solver initialization',
        'Asymmetric calculation with solver initialization',
        'Asymmetric calculation without solver initialization',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

comparison_df = pd.DataFrame(
    np.zeros(shape=(6, 2), dtype=np.float64),
    columns=['Deviation Voltage (p.u.)', 'Deviation Loading (p.u.)'],
    index=[
        'Symmetric calculation',
        'Asymmetric calculation',
        'Time series symmetric calculation',
        'Time series asymmetric calculation',
        'N-1 symmetric calculation',
        'N-1 asymmetric calculation',
    ]
)

## Simulation Parameters

The simulation parameters, for example, the total number of feeders `n_feeder`, are defined below.

In [3]:
# fictional grid parameters

n_node_per_feeder = 10
n_feeder = 100

cable_length_km_min = 0.8
cable_length_km_max = 1.2
load_p_w_max = 0.4e6 * 0.8
load_p_w_min = 0.4e6 * 1.2
pf = 0.95

load_scaling_min = 0.5
load_scaling_max = 1.5
n_step = 100
n_node = n_node_per_feeder * n_feeder + 1
n_line = n_node_per_feeder * n_feeder
n_load = n_node_per_feeder * n_feeder

## Pre-cache Library

To make a fair comparison, we run one small network so that both `power-grid-model` and `pandapower` can cache their dependent libraries into the memory.

* For `power-grid-model` the `MKL` `PARDISO` solver is loaded during the pre-cache.
* For `pandapower` the `numba` functions are JIT compiled and cached in the memory.

In [4]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=3,
    n_feeder=2,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp.runpp(fictional_dataset['pp_net'], algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
pgm_model = pgm.PowerGridModel(fictional_dataset['pgm_dataset'])
pgm_result = pgm_model.calculate_power_flow()

# Single Calculation

We begin with a single power flow calculation. First generate the fictional datasets.

In [5]:
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

## Symmetric

### Linear Method of power-grid-model

In [6]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear')
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [7]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow()
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [8]:
# first calculation with solver initialization
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Symmetric calculation without solver initialization', 'PandaPower NR'] = end - start

### Calculate Deviation for Newton-Raphson Method

In [9]:
comparison_df.loc['Symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus['vm_pu'] - pgm_result['node']['u_pu']).max()
comparison_df.loc['Symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

## Asymmetric

### Linear Method of power-grid-model

In [10]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM Linear'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear')
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [11]:
# first calculation with solver initialization
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PGM NR'] = end - start

# second calculation with existing solver
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [12]:
# first calculation with solver initialization
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation with solver initialization', 'PandaPower NR'] = end - start

# second calculation with existing solver
start = time.time()
pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
end = time.time()
summary_df.loc['Asymmetric calculation without solver initialization', 'PandaPower NR'] = end - start


### Calculate Deviation for Newton-Raphson Method

In [13]:
comparison_df.loc['Asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']].to_numpy() - pgm_result['node']['u_pu']).max()
comparison_df.loc['Asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_net.res_line_3ph['loading_percent'] * 1e-2 - pgm_result['line']['loading']).max()

# Time Series Calculation

We execute a time-series power flow with `n_step` timestamps. 

## Preparation

The load profile is randomly generated by the function `generate_time_series`. It produces the relevant input format for both libraries.

In [14]:
time_series_dataset = generate_time_series(
    fictional_dataset,
    n_step=n_step,
    load_scaling_min=load_scaling_min,
    load_scaling_max=load_scaling_max
)

pgm_update_dataset = time_series_dataset['pgm_update_dataset']
pp_dataset = time_series_dataset['pp_dataset']
time_steps = np.arange(n_step)

for x, y in zip(['p', 'q'], ['mw', 'mvar']):
    for p in ['a', 'b', 'c']:
        name = f'{x}_{p}_{y}'
        pp.control.ConstControl(
            pp_net,
            element='asymmetric_load',
            element_index=pp_net.asymmetric_load.index,
            variable=name,
            data_source=pp_dataset[name],
            profile_name=pp_net.asymmetric_load.index
        )


## Symmetric

### Linear Method of power-grid-model

In [15]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [16]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [17]:
pp.timeseries.OutputWriter(
    pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',',
    log_variables=[
        ('res_bus', 'vm_pu'),
        ('res_line', 'loading_percent'),
    ]
)

start = time.time()
pp.timeseries.run_timeseries(
    pp_net, run=pp.runpp, time_steps=time_steps,
    calculate_voltage_angles=True, distributed_slack=True
)
end = time.time()
summary_df.loc['Time series symmetric calculation', 'PandaPower NR'] = end - start

Progress: |██████████████████████████████████████████████████| 100.0% Complete



### Calculate Deviation for Newton-Raphson Method

In [18]:
pp_u_pu = pd.read_csv(output_dir / 'res_bus' / 'vm_pu.csv', index_col=0).to_numpy()
comparison_df.loc['Time series symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
pp_loading = pd.read_csv(output_dir / 'res_line' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
comparison_df.loc['Time series symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()


## Asymmetric

### Linear Method of power-grid-model

In [19]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [20]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(symmetric=False, update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [21]:
del pp_net.output_writer
ow = pp.timeseries.OutputWriter(pp_net, output_path=output_dir, output_file_type=".csv", csv_separator=',')
ow.log_variable('res_bus_3ph', 'vm_a_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_b_pu', index=pp_net.bus.index)
ow.log_variable('res_bus_3ph', 'vm_c_pu', index=pp_net.bus.index)
ow.log_variable('res_line_3ph', 'loading_percent', index=pp_net.line.index)

# run
start = time.time()
pp.timeseries.run_timeseries(
    pp_net,
    run=pp.runpp_3ph,
    time_steps=time_steps,
    calculate_voltage_angles=True,
    distributed_slack=True
)
end = time.time()
summary_df.loc['Time series asymmetric calculation', 'PandaPower NR'] = end - start

Progress: |██████████████████████████████████████████████████| 100.0% Complete



### Calculate Deviation for Newton-Raphson Method

In [22]:
pp_u_pu = []
for p in ['a', 'b', 'c']:
    pp_u_pu.append(pd.read_csv(output_dir / 'res_bus_3ph' / f'vm_{p}_pu.csv', index_col=0).to_numpy())
pp_u_pu = np.stack(pp_u_pu, axis=-1)
comparison_df.loc['Time series asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
pp_loading = pd.read_csv(output_dir / 'res_line_3ph' / 'loading_percent.csv', index_col=0).to_numpy() * 1e-2
comparison_df.loc['Time series asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

# N-1 Scenario Calculation

We execute a N-1 scenario calculation. There are `n_line` scenarios. In each scenario one line will be disabled. Since the original network is radial, part of the network will be unenergized due to the switch-off of a line.

## Preparation

The N-1 scenario input is generated for `power-grid-model`. Since `pandapower` does not have a built-in N-1 scenario calculation, we execute the power flow per scenarion in a loop.

In [23]:
# re-generate dataset
fictional_dataset = generate_fictional_grid(
    n_node_per_feeder=n_node_per_feeder,
    n_feeder=n_feeder,
    cable_length_km_min=cable_length_km_min,
    cable_length_km_max=cable_length_km_max,
    load_p_w_max=load_p_w_max,
    load_p_w_min=load_p_w_min,
    pf=pf
)

pp_net = fictional_dataset['pp_net']
pgm_dataset = fictional_dataset['pgm_dataset']

# update dataset for power grid model
# disable one line per batch
pgm_line_profile = pgm.initialize_array('update', 'line', (n_line, n_line))
pgm_line_profile['id'] = pgm_dataset['line']['id']
pgm_line_profile['from_status'] = 1
pgm_line_profile['to_status'] = 1
np.fill_diagonal(pgm_line_profile['from_status'], 0)
np.fill_diagonal(pgm_line_profile['to_status'], 0)
pgm_update_dataset = {'line': pgm_line_profile}




## Symmetric

### Linear Method of power-grid-model

In [24]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['N-1 symmetric calculation', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [25]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset)
end = time.time()
summary_df.loc['N-1 symmetric calculation', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [26]:
# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, :] = pp_net.res_bus['vm_pu']
    pp_loading[i, :] = pp_net.res_line['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 symmetric calculation', 'PandaPower NR'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

### Calculate Deviation for Newton-Raphson Method

In [27]:
comparison_df.loc['N-1 symmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 symmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

## Asymmetric

### Linear Method of power-grid-model

In [28]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(calculation_method='linear', update_data=pgm_update_dataset, symmetric=False)
end = time.time()
summary_df.loc['N-1 asymmetric calculation', 'PGM Linear'] = end - start

### Newton-Raphson Method of power-grid-model

In [29]:
pgm_model = pgm.PowerGridModel(pgm_dataset)
start = time.time()
pgm_result = pgm_model.calculate_power_flow(update_data=pgm_update_dataset, symmetric=False)
end = time.time()
summary_df.loc['N-1 asymmetric calculation', 'PGM NR'] = end - start

### Newton-Raphson Method of pandapower

In [30]:
# prepare pandapower result dataset
pp_u_pu = np.empty(shape=(n_line, n_node, 3), dtype=np.float64)
pp_loading = np.empty(shape=(n_line, n_line), dtype=np.float64)

start = time.time()
# loop to calcualte pandapower N-1
for i in pp_net.line.index:
    # set one line out of service
    pp_net.line.loc[i, 'in_service'] = False
    pp.runpp_3ph(pp_net, algorithm='nr', calculate_voltage_angles=True, distributed_slack=True)
    # restore that line
    pp_net.line.loc[i, 'in_service'] = True
    # get result
    pp_u_pu[i, ...] = pp_net.res_bus_3ph[['vm_a_pu', 'vm_b_pu', 'vm_c_pu']]
    pp_loading[i, :] = pp_net.res_line_3ph['loading_percent'] * 1e-2
end = time.time()
summary_df.loc['N-1 asymmetric calculation', 'PandaPower NR'] = end - start

# set nan to 0.0 to make a meaningful comparison
pp_u_pu[np.isnan(pp_u_pu)] = 0.0
pp_loading[np.isnan(pp_loading)] = 0.0

c:\users\al22994\workspace\power-grid-model-benchmark\py\lib\site-packages\pandapower\results_bus.py:54: RuntimeWarning: invalid value encountered in true_divide
  net["res_bus_3ph"]["unbalance_percent"] = np.abs(V012_pu[2, :]/V012_pu[1, :])*100
c:\users\al22994\workspace\power-grid-model-benchmark\py\lib\site-packages\pandapower\auxiliary.py:838: RuntimeWarning: invalid value encountered in true_divide
  return np.conjugate(np.divide(S, V, out=np.zeros_like(S), where=V != 0))  # Return zero if div by zero


### Calculate Deviation for Newton-Raphson Method

In [31]:
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Voltage (p.u.)'] = \
    np.abs(pp_u_pu - pgm_result['node']['u_pu']).max()
comparison_df.loc['N-1 asymmetric calculation', 'Deviation Loading (p.u.)'] = \
    np.abs(pp_loading - pgm_result['line']['loading']).max()

# Summary

## Deviation of Results

Below is the table of deviation between the results from `power-grid-model` and `pandapower`. It matches to the order of `1e-7`.

In [32]:
display(comparison_df)

,Deviation Voltage (p.u.),Deviation Loading (p.u.)
Symmetric calculation,1.626921e-12,5.124789e-13
Asymmetric calculation,3.299400e-08,1.606717e-07
Time series symmetric calculation,1.701528e-12,7.985834e-13
Time series asymmetric calculation,1.253202e-07,5.417755e-07
N-1 symmetric calculation,1.638023e-12,6.037393e-13
N-1 asymmetric calculation,3.299403e-08,1.606718e-07


## Performance Comparison (Windows)

Below is the table of the measured time of all calculations.

In [33]:
display(summary_df)

,PGM Linear,PGM NR,PandaPower NR
Symmetric calculation with solver initialization,0.003014,0.005001,0.021000
Symmetric calculation without solver initialization,0.000965,0.004994,0.016000
Asymmetric calculation with solver initialization,0.003998,0.019996,0.114041
Asymmetric calculation without solver initialization,0.003001,0.018034,0.109637
Time series symmetric calculation,0.149981,0.462623,2.120596
Time series asymmetric calculation,0.322160,2.001226,13.486438
N-1 symmetric calculation,1.439919,4.552637,14.583385
N-1 asymmetric calculation,3.174342,19.774518,108.292559


In [34]:
relative_df = summary_df.div(summary_df.max(axis=1), axis=0)
speedup_df = 1 / relative_df
display(speedup_df.style.format("{:0.2f}x"))

,PGM Linear,PGM NR,PandaPower NR
Symmetric calculation with solver initialization,6.97x,4.20x,1.00x
Symmetric calculation without solver initialization,16.58x,3.20x,1.00x
Asymmetric calculation with solver initialization,28.53x,5.70x,1.00x
Asymmetric calculation without solver initialization,36.53x,6.08x,1.00x
Time series symmetric calculation,14.14x,4.58x,1.00x
Time series asymmetric calculation,41.86x,6.74x,1.00x
N-1 symmetric calculation,10.13x,3.20x,1.00x
N-1 asymmetric calculation,34.11x,5.48x,1.00x


## Cached Results Linux (WSL)

In [35]:
cached_wsl_df = pd.DataFrame.from_dict(
    {
        "Symmetric with solver initialization": (0.003339, 0.004976, 0.024358),
        "Symmetric without solver initialization": (0.002114, 0.004114, 0.017295),
        "Asymmetric with solver initialization": (0.004563, 0.010456, 0.204329),
        "Asymmetric without solver initialization": (0.002258, 0.010334, 0.179339),
        "Time series symmetric calculation": (0.166692, 0.498151, 1.974510),
        "Time series asymmetric calculation": (0.241750, 0.905349, 22.923779),
        "N-1 symmetric calculation": (1.395829, 4.573187, 14.114142),
        "N-1 asymmetric calculation": (2.569487, 9.330348, 198.132612),
    },
    columns=["PGM Linear Linux", "PGM NR Linux", "PandaPower NR Linux"],
    orient="index"
)
display(cached_wsl_df)

,PGM Linear Linux,PGM NR Linux,PandaPower NR Linux
Symmetric with solver initialization,0.003339,0.004976,0.024358
Symmetric without solver initialization,0.002114,0.004114,0.017295
Asymmetric with solver initialization,0.004563,0.010456,0.204329
Asymmetric without solver initialization,0.002258,0.010334,0.179339
Time series symmetric calculation,0.166692,0.498151,1.974510
Time series asymmetric calculation,0.241750,0.905349,22.923779
N-1 symmetric calculation,1.395829,4.573187,14.114142
N-1 asymmetric calculation,2.569487,9.330348,198.132612


In [36]:
relative_df = cached_wsl_df.div(cached_wsl_df.max(axis=1), axis=0)
speedup_df = 1 / relative_df
display(speedup_df.style.format("{:0.2f}x"))

,PGM Linear Linux,PGM NR Linux,PandaPower NR Linux
Symmetric with solver initialization,7.29x,4.90x,1.00x
Symmetric without solver initialization,8.18x,4.20x,1.00x
Asymmetric with solver initialization,44.78x,19.54x,1.00x
Asymmetric without solver initialization,79.42x,17.35x,1.00x
Time series symmetric calculation,11.85x,3.96x,1.00x
Time series asymmetric calculation,94.82x,25.32x,1.00x
N-1 symmetric calculation,10.11x,3.09x,1.00x
N-1 asymmetric calculation,77.11x,21.24x,1.00x
